## 문제
<br>JOI국은 N개의 도시와 M개의 도로로 이루어져 있다. 모든 도시는 도로로 연결되어 있으며, 각 도로를 통하지 않고는 다른 도시로 갈 수 없다.

<br>이번에 K개의 도시는 좀비에 의해서 점령당했다. ㅠㅠ

<br>따라서 경곽이는 벙커가 있는 가장 안전한 도시로 피난을 가기로 했다. <br>경곽이는 현재 1번 도시에 살고 있으며, 벙커가 있는 가장 안전한 피난처는 N번 도시이다. <br>1번 도시와 N번 도시는 아직 좀비에게 점령당하지 않았다.

<br>경곽이는 각 도시를 이동할 때마다 1박을 해야하고, 1박을 할 때 숙박비를 지불해야 한다. <br>만약 그 도시가 좀비에게 점령당했다면 숙박이 불가능하다.

<br>좀비에게 점령당한 도시로 부터 S번 이하의 이동으로 이동할 수 있는 모든 도시는 위험한 도시로 정의하며, <br>그 이외의 도시는 안전한 도시로 정의할 때, 만약 그 도시가 안전한 도시라면 숙박비가 p원이고, 만약 그 도시가 위험한 도시라면 숙박비는 q원이다. <br>(좀비로부터 보호받기 위한 특별한 시큐리티 서비스를 제공하기 때문에 좀 더 비싸다 ㅠㅠ)

<br>경곽이가 도시 1부터 N으로 이동하는 데 드는 최단 비용을 구하자.

In [ ]:
n, m, k, s = map(int, input().split()) # n:도시, m:도로, k:점령당한도시, s번이하이동
p, q = map(int, input().split()) # p:안전숙박비, q:위험숙박비

## 입력
<br>첫 번째 줄에 N, M, K, S가 공백으로 구분되어 입력된다. 각 값은 도시의 수, 길의 수, 좀비에게 점령당한 도시의 수, 위험한 도시의 범위 를 의미한다. 
<br>(2 ≦ N ≦ 100000, 1 ≦ M ≦ 200000, 0 ≦ K ≦ N - 2, 0 ≦ S ≦ 100000)

<br>다음 줄에는 숙박비를 나타내는 정수 p, q가 입력된다. (1 ≦ p ＜ q ≦ 100000)

<br>그 다음 줄부터 K줄에 걸쳐서 좀비에게 점령당한 도시가 한 줄에 하나씩 주어진다.

<br>다음 줄부터 M줄에 걸쳐서 각 도시를 연결하는 도로의 정보가 주어진다. 이 도로는 서로 양방향으로 이동 가능하다.

<br>1번 도시에서 N번 도시로 항상 도달 가능하다.

## 출력
<br>최소 비용을 출력한다.

### 예제 입력 1 
<br>13 21 1 1
<br>1000 6000
<br>7
<br>1 2
<br>3 7
<br>2 4
<br>5 8
<br>8 9
<br>2 5
<br>3 4
<br>4 7
<br>9 10
<br>10 11
<br>5 9
<br>7 12
<br>3 6
<br>4 5
<br>1 3
<br>11 12
<br>6 7
<br>8 11
<br>6 13
<br>7 8
<br>12 13
### 예제 출력 1 
<br>11000

In [14]:
import copy
from collections import deque # BFS
import heapq # 다익스트라 알고리즘

# input
N, M, K, S = map(int, input().split()) # n:도시, m:도로, k:점령당한도시, s번이하이동
P, Q = map(int, input().split()) # p:안전숙박비12 , q:위험숙박비
occupied = [int(input(i)) for i in range(K)] # 점령된 도시들
edges = [list(map(int, input().split())) for _ in range(M)] # 도로들

In [17]:
edges_copy = copy.deepcopy(edges)

In [18]:
# 인접 리스트로 무향 그래프 생성하기
def make_graph(): # graph[i] = 도시 i에 연결된 다른 도시들 저장 리스트 
    graph = [[] for _ in range(N + 1)] # 0번 인덱스는 사용하지 않을 예정(도시는 1번부터 N번까지 존재)
    for frm, to in edges:
        graph[frm].append(to) # "왼쪽이 오른쪽에 연결되어 있어요"
        graph[to].append(frm) # "오른쪽이 왼쪽에 연결되어 있어요"
    return graph

In [ ]:
# 좀비 도시 + 위험한 도시 찾기
def find_zombie_land(graph): # 이 그래프에서 위험 지역을 찾아라~!
    zombie_land = [0] * (N + 1) # 마찬가지로 0번째 인덱스는 버린다
        # 모든 땅번호에 대해 0으로 초기화하고 
    
    for start in occupied: # 모든 좀비도시들을 시작점으로 해서 
        lands = bfs(start, graph, S) # S거리 이내의 위험지역을 찾는 BFS를 수행
        for land in lands: # 그 위험지역들에 대해서는 
            zombie_land[land] = 1 # 1값을 저장 

    zombie_land[1] = 0 # 출발점은 당연히 0 
    zombie_land[N] = 0 # 목표 도착점은 당연히 0 

    return zombie_land # 위험+좀비도시에 1표시해둔 리스트(나머지는0)

In [ ]:
# S거리 이내의 위험 도시 탐색을 위한 BFS
def bfs(start, graph, offset): # 출발점, 그래프(인접리스트), 위험 판단 거리 1, graph, 1
    visit = [0] * (N + 1) #방문여부  #[0 0 0 ... 0]
    q = deque() # []

    ret = [] # 위험지역 번호 저장할 리스트 초기화 # []
    visit[start] = 1 # 출발점 방문 처리 # [0 1 0 .. 0]
    q.append([start, 0]) # 큐에 시작 도시 + 현재 이동 거리 삽입 # q:[[1,0]]

    while q: # q가 비지 않았을 때 q:[[1,0]]
        here, move = q.popleft() # 현재 이동 후보위치(1), 현재까지 이동한 거리(0)q:[]

        if move > offset: # 위험판단거리 이상이면 무시 # 밑의 for 문도 시행되지 X 
            continue # q가 빈채로 남아있게 됨 
        else: 
            ret.append(here) # 아니라면(기준보다 가깝다면) 현재위치 번호를 위험 위치에 추가

        for there in graph[here]: # 현재 위치와 연결되어 있는 노드들에 대해, 
            if not visit[there]: # 그 노드들 중 방문하지 않은 노드들을 모두 
                visit[there] = 1 # 방문표시 하고
                q.append([there, move + 1]) # 거길 방문하고, move에 1을 더해줌 

    return ret

# 다익스트라 알고리즘을 사용한 최소 비용 계산 
def dijkstra(graph, zombie_land):
    pq = []
    dist = [sys.maxsize] * (N + 1)

    dist[1] = 0
    heapq.heappush(pq, [0, 1])

    while pq:
        cost, here = heapq.heappop(pq)

        if dist[here] < cost:
            continue

        for there in graph[here]:
            if there in occupied:
                continue

            next_cost = cost + (Q if zombie_land[there] else P)
            if next_cost < dist[there]:
                dist[there] = next_cost
                heapq.heappush(pq, [next_cost, there])

    return dist[N] - P

# 전체 솔루션 함수
def solution():
    graph = make_graph()
    zombie_land = find_zombie_land(graph)
    answer = dijkstra(graph, zombie_land)

    return answer

# 결과 출력
print(solution())


# 최종 Solution

In [1]:
from collections import deque # BFS
import heapq # 다익스트라 알고리즘

# input
N, M, K, S = map(int, input().split()) # n:도시, m:도로, k:점령당한도시, s번이하이동
P, Q = map(int, input().split()) # p:안전숙박비12 , q:위험숙박비
occupied = [int(input(i)) for i in range(K)] # 점령된 도시들
edges = [list(map(int, input().split())) for _ in range(M)] # 도로들

# 인접 리스트로 무향 그래프 생성하기
def make_graph(): # graph[i] = 도시 i에 연결된 다른 도시들 저장 리스트 
    graph = [[] for _ in range(N + 1)] # 0번 인덱스는 사용하지 않을 예정(도시는 1번부터 N번까지 존재)
    for frm, to in edges:
        graph[frm].append(to) # "왼쪽이 오른쪽에 연결되어 있어요"
        graph[to].append(frm) # "오른쪽이 왼쪽에 연결되어 있어요"
    return graph

# 좀비 도시 + 위험한 도시 찾기
def find_zombie_land(graph): # 이 그래프에서 위험 지역을 찾아라~!
    zombie_land = [0] * (N + 1) # 마찬가지로 0번째 인덱스는 버린다
        # 모든 땅번호에 대해 0으로 초기화하고 
    
    for start in occupied: # 모든 좀비도시들을 시작점으로 해서 
        lands = bfs(start, graph, S) # S거리 이내의 위험지역을 찾는 BFS를 수행
        for land in lands: # 그 위험지역들에 대해서는 
            zombie_land[land] = 1 # 1값을 저장 

    zombie_land[1] = 0 # 출발점은 당연히 0 
    zombie_land[N] = 0 # 목표 도착점은 당연히 0 

    return zombie_land # 위험+좀비도시에 1표시해둔 리스트(나머지는0)

# S거리 이내의 위험 도시 탐색을 위한 BFS
def bfs(start, graph, offset): # 출발점, 그래프(인접리스트), 위험 판단 거리 1, graph, 1
    visit = [0] * (N + 1) #방문여부  #[0 0 0 ... 0]
    q = deque() # []

    ret = [] # 위험지역 번호 저장할 리스트 초기화 # []
    visit[start] = 1 # 출발점 방문 처리 # [0 1 0 .. 0]
    q.append([start, 0]) # 큐에 시작 도시 + 현재 이동 거리 삽입 # q:[[1,0]]

    while q: # q가 비지 않았을 때 q:[[1,0]]
        here, move = q.popleft() # 현재 이동 후보위치(1), 현재까지 이동한 거리(0)q:[]

        if move > offset: # 위험판단거리 이상이면 무시 # 밑의 for 문도 시행되지 X 
            continue # q가 빈채로 남아있게 됨 
        else: 
            ret.append(here) # 아니라면(기준보다 가깝다면) 현재위치 번호를 위험 위치에 추가

        for there in graph[here]: # 현재 위치와 연결되어 있는 노드들에 대해, 
            if not visit[there]: # 그 노드들 중 방문하지 않은 노드들을 모두 
                visit[there] = 1 # 방문표시 하고
                q.append([there, move + 1]) # 거길 방문하고, move에 1을 더해줌 

    return ret

# 다익스트라 알고리즘을 사용한 최소 비용 계산 
def dijkstra(graph, zombie_land):
    pq = []
    dist = [sys.maxsize] * (N + 1)

    dist[1] = 0
    heapq.heappush(pq, [0, 1])

    while pq:
        cost, here = heapq.heappop(pq)

        if dist[here] < cost:
            continue

        for there in graph[here]:
            if there in occupied:
                continue

            next_cost = cost + (Q if zombie_land[there] else P)
            if next_cost < dist[there]:
                dist[there] = next_cost
                heapq.heappush(pq, [next_cost, there])

    return dist[N] - P

# 전체 솔루션 함수
def solution():
    graph = make_graph()
    zombie_land = find_zombie_land(graph)
    answer = dijkstra(graph, zombie_land)

    return answer

# 결과 출력
print(solution())

11000
